# Inspect and summarize SLURM benchmarks

## Overview

## Load, merge and compare benchmark summary `csv`s

We need a function to compare our SLURM summary output files and highlight differences in a table.

In [1]:
library(dplyr)
library(tidyr)

read_and_summarize_differences <- function(directory) {
  # List all CSV files in the directory
  files <- list.files(directory, pattern = "\\.csv$", full.names = TRUE)
  files <- files[!grepl("1000000", files)]
  
  # Read each file and store in a list
  data_list <- lapply(files, function(file) {
    data <- read.csv(file)
    # Add a column to identify the source file
    data$Source <- basename(file)
    return(data)
  })
  
  # Combine all data frames into one
  combined_data <- bind_rows(data_list)
  
  # Spread the data to wide format for comparison
  wide_data <- spread(combined_data, key = Parameter, value = Value)
  
  # Identify rows with at least one difference in values
  differing_rows <- wide_data %>%
    summarise(across(-Source, ~n_distinct(.) > 1)) %>%
    select(which(colSums(.) > 0)) %>%
    names()
  
  # Filter the combined data to include only differing parameters
  summary_data <- combined_data %>%
    filter(Parameter %in% differing_rows) %>%
    select(Parameter, Source, Value)
  
  # Pivot wider for a clearer summary table, if necessary
  summary_table <- summary_data %>%
    pivot_wider(names_from = Source, values_from = Value)
  
  return(summary_table)
}


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
directory <- "/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_chr1_a1/"
summary_table <- read_and_summarize_differences(directory)
summary_table

Parameter,libd_chr1-chr1_AA-libd_chr1-chr1_AA-108982-128981-dynamic-1corestotal-allcorepera-20240409-110212-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-128982-148981-dynamic-1corestotal-allcorepera-20240409-110348-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-28982-48981-dynamic-1corestotal-allcorepera-20240409-110818-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-48982-68981-dynamic-1corestotal-allcorepera-20240409-105727-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-68982-88981-dynamic-1corestotal-allcorepera-20240409-105902-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-88982-108981-dynamic-1corestotal-allcorepera-20240409-110037-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-8982-28981-dynamic-1corestotal-allcorepera-20240409-105417-summary.csv
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
chunk1,108982,128982,28982,48982,68982,88982,8982
chunk2,128981,148981,48981,68981,88981,108981,28981
tag,libd_chr1-chr1_AA-108982-128981-dynamic-1corestotal-allcorepera-20240409-110212,libd_chr1-chr1_AA-128982-148981-dynamic-1corestotal-allcorepera-20240409-110348,libd_chr1-chr1_AA-28982-48981-dynamic-1corestotal-allcorepera-20240409-110818,libd_chr1-chr1_AA-48982-68981-dynamic-1corestotal-allcorepera-20240409-105727,libd_chr1-chr1_AA-68982-88981-dynamic-1corestotal-allcorepera-20240409-105902,libd_chr1-chr1_AA-88982-108981-dynamic-1corestotal-allcorepera-20240409-110037,libd_chr1-chr1_AA-8982-28981-dynamic-1corestotal-allcorepera-20240409-105417
time_started,2024-04-09 11:02:55,2024-04-09 11:03:57,2024-04-09 11:09:09,2024-04-09 10:57:48,2024-04-09 10:59:50,2024-04-09 11:00:52,2024-04-09 10:54:44
time_finished,2024-04-09 21:33:59,2024-04-09 19:53:19,2024-04-09 14:00:31,2024-04-09 16:32:57,2024-04-09 18:40:47,2024-04-10 03:48:06,2024-04-09 13:58:46
runtime,10:31:04,08:49:22,02:51:22,05:35:08,07:40:56,16:47:14,03:04:02
amount_RAM,0.245569720864296,0.245569724589586,0.245569720864296,0.245569724589586,0.245569720864296,0.245569720864296,0.245569720864296
scaffold_ID,libd_chr1-chr1_AA-libd_chr1-chr1_AA-108982-128981-dynamic-1corestotal-allcorepera-20240409-110212,libd_chr1-chr1_AA-libd_chr1-chr1_AA-128982-148981-dynamic-1corestotal-allcorepera-20240409-110348,libd_chr1-chr1_AA-libd_chr1-chr1_AA-28982-48981-dynamic-1corestotal-allcorepera-20240409-110818,libd_chr1-chr1_AA-libd_chr1-chr1_AA-48982-68981-dynamic-1corestotal-allcorepera-20240409-105727,libd_chr1-chr1_AA-libd_chr1-chr1_AA-68982-88981-dynamic-1corestotal-allcorepera-20240409-105902,libd_chr1-chr1_AA-libd_chr1-chr1_AA-88982-108981-dynamic-1corestotal-allcorepera-20240409-110037,libd_chr1-chr1_AA-libd_chr1-chr1_AA-8982-28981-dynamic-1corestotal-allcorepera-20240409-105417


## Compute core-hours per test

In [3]:
windows <- c(10000)
n_windows <- length(windows)
alphas <- seq(0.5) # left out 0 on first run
n_alphas <- length(alphas)
#chunks <- 10000
#n_tests <- n_windows * n_alphas * chunks
#n_tests

In [4]:
alphas

[1] 1

In [5]:
colnames(summary_table)

[1] "Parameter"                                                                                                    
[2] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-108982-128981-dynamic-1corestotal-allcorepera-20240409-110212-summary.csv"
[3] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-128982-148981-dynamic-1corestotal-allcorepera-20240409-110348-summary.csv"
[4] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-28982-48981-dynamic-1corestotal-allcorepera-20240409-110818-summary.csv"  
[5] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-48982-68981-dynamic-1corestotal-allcorepera-20240409-105727-summary.csv"  
[6] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-68982-88981-dynamic-1corestotal-allcorepera-20240409-105902-summary.csv"  
[7] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-88982-108981-dynamic-1corestotal-allcorepera-20240409-110037-summary.csv" 
[8] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-8982-28981-dynamic-1corestotal-allcorepera-20240409-105417-summary.csv"

In [6]:
summary_table

Parameter,libd_chr1-chr1_AA-libd_chr1-chr1_AA-108982-128981-dynamic-1corestotal-allcorepera-20240409-110212-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-128982-148981-dynamic-1corestotal-allcorepera-20240409-110348-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-28982-48981-dynamic-1corestotal-allcorepera-20240409-110818-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-48982-68981-dynamic-1corestotal-allcorepera-20240409-105727-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-68982-88981-dynamic-1corestotal-allcorepera-20240409-105902-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-88982-108981-dynamic-1corestotal-allcorepera-20240409-110037-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-8982-28981-dynamic-1corestotal-allcorepera-20240409-105417-summary.csv
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
chunk1,108982,128982,28982,48982,68982,88982,8982
chunk2,128981,148981,48981,68981,88981,108981,28981
tag,libd_chr1-chr1_AA-108982-128981-dynamic-1corestotal-allcorepera-20240409-110212,libd_chr1-chr1_AA-128982-148981-dynamic-1corestotal-allcorepera-20240409-110348,libd_chr1-chr1_AA-28982-48981-dynamic-1corestotal-allcorepera-20240409-110818,libd_chr1-chr1_AA-48982-68981-dynamic-1corestotal-allcorepera-20240409-105727,libd_chr1-chr1_AA-68982-88981-dynamic-1corestotal-allcorepera-20240409-105902,libd_chr1-chr1_AA-88982-108981-dynamic-1corestotal-allcorepera-20240409-110037,libd_chr1-chr1_AA-8982-28981-dynamic-1corestotal-allcorepera-20240409-105417
time_started,2024-04-09 11:02:55,2024-04-09 11:03:57,2024-04-09 11:09:09,2024-04-09 10:57:48,2024-04-09 10:59:50,2024-04-09 11:00:52,2024-04-09 10:54:44
time_finished,2024-04-09 21:33:59,2024-04-09 19:53:19,2024-04-09 14:00:31,2024-04-09 16:32:57,2024-04-09 18:40:47,2024-04-10 03:48:06,2024-04-09 13:58:46
runtime,10:31:04,08:49:22,02:51:22,05:35:08,07:40:56,16:47:14,03:04:02
amount_RAM,0.245569720864296,0.245569724589586,0.245569720864296,0.245569724589586,0.245569720864296,0.245569720864296,0.245569720864296
scaffold_ID,libd_chr1-chr1_AA-libd_chr1-chr1_AA-108982-128981-dynamic-1corestotal-allcorepera-20240409-110212,libd_chr1-chr1_AA-libd_chr1-chr1_AA-128982-148981-dynamic-1corestotal-allcorepera-20240409-110348,libd_chr1-chr1_AA-libd_chr1-chr1_AA-28982-48981-dynamic-1corestotal-allcorepera-20240409-110818,libd_chr1-chr1_AA-libd_chr1-chr1_AA-48982-68981-dynamic-1corestotal-allcorepera-20240409-105727,libd_chr1-chr1_AA-libd_chr1-chr1_AA-68982-88981-dynamic-1corestotal-allcorepera-20240409-105902,libd_chr1-chr1_AA-libd_chr1-chr1_AA-88982-108981-dynamic-1corestotal-allcorepera-20240409-110037,libd_chr1-chr1_AA-libd_chr1-chr1_AA-8982-28981-dynamic-1corestotal-allcorepera-20240409-105417


In [7]:
summary_table <- as.data.frame(t(summary_table))
colnames(summary_table) <- summary_table[1, ]
summary_table <- summary_table[-1, ]

In [8]:
#summary_table$number_cores <- as.numeric(as.character(summary_table$num_cores))
summary_table$number_cores <- 1

In [9]:
summary_table$runtime_seconds <- sapply(summary_table$runtime, function(x) {
  parts <- strsplit(x, ":")[[1]]
  as.numeric(parts[1]) * 3600 + as.numeric(parts[2]) * 60 + as.numeric(parts[3])
})

In [10]:
summary_table$runtime_hours <- summary_table$runtime_seconds/3600

In [11]:
summary_table$chunk1 <- as.numeric(as.character(summary_table$chunk1))
summary_table$chunk2 <- as.numeric(as.character(summary_table$chunk2))

In [12]:
summary_table$runtime_core_hours <- summary_table$runtime_hours * summary_table$number_cores
summary_table$runtime_core_seconds <- summary_table$runtime_seconds * summary_table$number_cores
summary_table$core_hours_per_test <- summary_table$runtime_core_hours / (summary_table$chunk2 - summary_table$chunk1 + 1)
summary_table$core_seconds_per_test <- summary_table$runtime_core_seconds / (summary_table$chunk2 - summary_table$chunk1 + 1)

In [13]:
summary_table

,chunk1,chunk2,tag,time_started,time_finished,runtime,amount_RAM,scaffold_ID,number_cores,runtime_seconds,runtime_hours,runtime_core_hours,runtime_core_seconds,core_hours_per_test,core_seconds_per_test
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
libd_chr1-chr1_AA-libd_chr1-chr1_AA-108982-128981-dynamic-1corestotal-allcorepera-20240409-110212-summary.csv,108982,128981,libd_chr1-chr1_AA-108982-128981-dynamic-1corestotal-allcorepera-20240409-110212,2024-04-09 11:02:55,2024-04-09 21:33:59,10:31:04,0.245569720864296,libd_chr1-chr1_AA-libd_chr1-chr1_AA-108982-128981-dynamic-1corestotal-allcorepera-20240409-110212,1,37864,10.517778,10.517778,37864,0.0005258889,1.8932
libd_chr1-chr1_AA-libd_chr1-chr1_AA-128982-148981-dynamic-1corestotal-allcorepera-20240409-110348-summary.csv,128982,148981,libd_chr1-chr1_AA-128982-148981-dynamic-1corestotal-allcorepera-20240409-110348,2024-04-09 11:03:57,2024-04-09 19:53:19,08:49:22,0.245569724589586,libd_chr1-chr1_AA-libd_chr1-chr1_AA-128982-148981-dynamic-1corestotal-allcorepera-20240409-110348,1,31762,8.822778,8.822778,31762,0.0004411389,1.5881
libd_chr1-chr1_AA-libd_chr1-chr1_AA-28982-48981-dynamic-1corestotal-allcorepera-20240409-110818-summary.csv,28982,48981,libd_chr1-chr1_AA-28982-48981-dynamic-1corestotal-allcorepera-20240409-110818,2024-04-09 11:09:09,2024-04-09 14:00:31,02:51:22,0.245569720864296,libd_chr1-chr1_AA-libd_chr1-chr1_AA-28982-48981-dynamic-1corestotal-allcorepera-20240409-110818,1,10282,2.856111,2.856111,10282,0.0001428056,0.5141
libd_chr1-chr1_AA-libd_chr1-chr1_AA-48982-68981-dynamic-1corestotal-allcorepera-20240409-105727-summary.csv,48982,68981,libd_chr1-chr1_AA-48982-68981-dynamic-1corestotal-allcorepera-20240409-105727,2024-04-09 10:57:48,2024-04-09 16:32:57,05:35:08,0.245569724589586,libd_chr1-chr1_AA-libd_chr1-chr1_AA-48982-68981-dynamic-1corestotal-allcorepera-20240409-105727,1,20108,5.585556,5.585556,20108,0.0002792778,1.0054
libd_chr1-chr1_AA-libd_chr1-chr1_AA-68982-88981-dynamic-1corestotal-allcorepera-20240409-105902-summary.csv,68982,88981,libd_chr1-chr1_AA-68982-88981-dynamic-1corestotal-allcorepera-20240409-105902,2024-04-09 10:59:50,2024-04-09 18:40:47,07:40:56,0.245569720864296,libd_chr1-chr1_AA-libd_chr1-chr1_AA-68982-88981-dynamic-1corestotal-allcorepera-20240409-105902,1,27656,7.682222,7.682222,27656,0.0003841111,1.3828
libd_chr1-chr1_AA-libd_chr1-chr1_AA-88982-108981-dynamic-1corestotal-allcorepera-20240409-110037-summary.csv,88982,108981,libd_chr1-chr1_AA-88982-108981-dynamic-1corestotal-allcorepera-20240409-110037,2024-04-09 11:00:52,2024-04-10 03:48:06,16:47:14,0.245569720864296,libd_chr1-chr1_AA-libd_chr1-chr1_AA-88982-108981-dynamic-1corestotal-allcorepera-20240409-110037,1,60434,16.787222,16.787222,60434,0.0008393611,3.0217
libd_chr1-chr1_AA-libd_chr1-chr1_AA-8982-28981-dynamic-1corestotal-allcorepera-20240409-105417-summary.csv,8982,28981,libd_chr1-chr1_AA-8982-28981-dynamic-1corestotal-allcorepera-20240409-105417,2024-04-09 10:54:44,2024-04-09 13:58:46,03:04:02,0.245569720864296,libd_chr1-chr1_AA-libd_chr1-chr1_AA-8982-28981-dynamic-1corestotal-allcorepera-20240409-105417,1,11042,3.067222,3.067222,11042,0.0001533611,0.5521


In [14]:
#summary_table[which.min(summary_table$core_hours_per_test), ]

In [15]:
data.table::fwrite(summary_table, "05-OUT_SLURM_benchmarks_a4_many_sites.csv")

## How long for whole dataset?

***This is for benchmarks with 500kb windows***

In [17]:
#summary_table <- summary_table[which(summary_table$num_cores == 1), ]

In [18]:
n_sites <- 26000000

In [19]:
summary_table$core_hours_per_test

[1] 0.0005258889 0.0004411389 0.0001428056 0.0002792778 0.0003841111
[6] 0.0008393611 0.0001533611

In [20]:
# Core hours for single run over methylation dataset (single brain region, single population dataset)

n_sites*mean(summary_table$core_hours_per_test)

[1] 10273.51

In [19]:
# Core hours for single run over methylation dataset (single brain region, single population dataset)

n_sites*mean(summary_table$core_hours_per_test)

[1] 12461.94

In [21]:
# Core hours for 3 x 3 brain regions x population datasets

brain_regions <- 3
populations <- 3
n_sites*mean(summary_table$core_hours_per_test)*brain_regions*populations

[1] 92461.57